<h1 align="center"><font size="5">Segmenting and Clustering Neighborhoods in Toronto - Week 3 Assignment</font></h1>

#### Importing the necessary packages

In [182]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Reading the Wikipage into a list

##### The table that is needed is encapsulated in tbody ags. and table rows are encapsulated in  tr tags. For each row, when we just take  the text, we will see that each value is in a newline of it's own. I will use \n to split the row into individual cells, but before doing that I am adding ", " before the newline character. This will come in handy when we need to group neighborhoods by postal code and Boroughs

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('tbody')

RowList = []

for rows in table.find_all('tr'):
    row = rows.text.replace('\n', ', \n')
    RowList.append(row.split('\n'))


#### Now, I am reading list I collected above into a data frame and assigning column names. Delete the empty rows and the one row that has column names. I am also dropping all rows where Borough is 'Not assigned'

In [183]:
df = pd.DataFrame(RowList, columns=['empty', 'Postcode','Borough','Neighbourhood','empty2'])
df.drop(columns=['empty','empty2'], inplace=True)
df.drop(0, inplace=True)
df.drop(df[df.Borough=='Not assigned, '].index, axis=0, inplace=True)
df.shape

(212, 3)

#### If Neighborhood is not assigned, then update Neighbourhood value to Borough value

In [191]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

#### Search for 'Not Assigned' Neighborhood and confirm it returns no results

In [192]:
df['Neighbourhood'].where(df['Neighbourhood'] == 'Not assigned')

3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
11     NaN
12     NaN
13     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
32     NaN
33     NaN
34     NaN
35     NaN
36     NaN
39     NaN
      ... 
237    NaN
238    NaN
241    NaN
242    NaN
245    NaN
246    NaN
247    NaN
248    NaN
251    NaN
252    NaN
253    NaN
254    NaN
257    NaN
258    NaN
259    NaN
264    NaN
267    NaN
268    NaN
269    NaN
270    NaN
271    NaN
272    NaN
273    NaN
274    NaN
275    NaN
284    NaN
285    NaN
286    NaN
287    NaN
288    NaN
Name: Neighbourhood, Length: 212, dtype: object

#### Add the Neighborhoods if Postcode and Borough are the same. The comma that we added above comes in handy now to separate different neighbourhoods in the same row. However, we need to strip the last character in each column as that is a comma.

In [189]:
Groupbydf = df.groupby(['Postcode','Borough']).sum()
newdf=Groupbydf.reset_index()
newdf.Postcode= newdf.Postcode.apply(lambda x: x[:-2])
newdf.Borough = newdf.Borough.apply(lambda x: x[:-2])
newdf.Neighbourhood = newdf.Neighbourhood.apply(lambda x: x[:-2])

newdf

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### The newcount below reflects the fact that mutliple neighborhoods are combined in one row.

In [190]:
newdf.shape

(103, 3)